In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.cuda.is_available()

True

In [ ]:
available_memory = torch.cuda.get_device_properties(0).total_memory
print(available_memory)

15828320256


In [ ]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 16e9:
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      torch_dtype=torch.float16,
      device_map="auto",
      use_cache=True
  )
else:
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      load_in_8bit=True,
      device_map="auto",
      use_cache=True
  )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
prompt = """### Task
Generate a SQL qury to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I don't know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplies by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products(
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10, 2), -- Price of each unit of the product
  quantity INTEGER -- Quantity of the product in stock
);

CREATE TABLE customers(
  customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
  name VARCHAR(50), -- Name of the customer
  address VARCHAR(100) -- Address of the customer
);

CREATE TABLE salespeople(
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales(
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of the product sold
  customer_id INTEGER, -- ID of the customer who made the purchase
  salesperson_id INTEGER, -- ID of the salesperson who made the sale
  sale_date DATE, -- Date of the sale occured
  quantity INTEGER -- Quantity of the product sold
);

CREATE TABLE product_suppliers(
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- product ID supplied
  supply_price DECIMAL(10, 2) -- unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""



In [ ]:
import sqlparse

def generate_query(question):
  updated_prompt = prompt.format(question=question)
  inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
  generated_ids = model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.pad_token_id,
      max_new_tokens=400,
      do_sample=False,
      num_beams=1,
  )
  outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  generated_sql = sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)
  print(generated_sql)

In [ ]:
question = "What was the highest quantity sold last month?"
generate_query(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



SELECT MAX(s.quantity) AS max_quantity_sold
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month');


In [ ]:
question = "Which salesperson sold the large amount of products in the last month?"
generate_query(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



SELECT s.salesperson_id,
       SUM(s.quantity) AS total_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY s.salesperson_id
ORDER BY total_quantity DESC NULLS LAST
LIMIT 1;
